### Download Local Model with Ollama

(1) Download Ollama:
https://ollama.com/download

(2) Select a model, for example Mistral 7B or Llama 3.1:
https://ollama.com/library/mistral:7b
https://ollama.com/library/llama3.1

(3) Download the model using command line:
ollama run mistral

In [1]:
import sys
import os
import copy
    
from typing import List, Dict
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display, HTML

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor, create_react_agent
from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama

from langchain_core.tools import tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

#from langchain.agents import Tool
import yfinance as yf
import pandas as pd

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
@tool
def get_stock_prices_and_volumes(symbol: str, start_date: str, end_date: str) -> str:
    """Get stock data."""
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(start=start_date, end=end_date)
        
        if data.empty:
            return f"No data found for {symbol} between {start_date} and {end_date}."
        
        data = data[['Open', 'High', 'Low', 'Close', 'Volume']]  
        summary = data.to_string()
        
        return f"Stock prices and volumes for {symbol} from {start_date} to {end_date}:\n\n{summary}"
    except Exception as e:
        return f"Error fetching stock prices and volumes for {symbol}: {str(e)}"

@tool
def calculate_var(symbol: str, start_date: str, end_date) -> str:
    """
    Calculates the 95% Value at Risk (VaR) using historical data.
    """
    try:
        # Fetch historical prices
        var_data = get_stock_prices_and_volumes(symbol, start_date, end_date)

        if var_data.empty:
            return f"No data found for {symbol} between {start_date} and {end_date} for VaR calculation."

        # Calculate daily returns from closing prices
        var_data['Returns'] = var_data['Close'].pct_change().dropna()

        # Compute the 95% VaR
        var = np.percentile(var_data['Returns'].dropna(), 5)

        return (
            f"The Value at Risk (VaR) for {symbol} "
            f"using data from {start_date} to {end_date} is approximately {var:.4%}."
        )
    except Exception as e:
        return f"Error calculating VaR for {symbol}: {str(e)}"

In [4]:
tools = [wiki, get_stock_prices_and_volumes, calculate_var]
llm = ChatOllama(
    #model="llama3.1",
    model='mistral:7b',
    temperature=0,
).bind_tools(tools)


In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to convert date to YYYY-MM-DD format.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)

In [ ]:
response = agent_executor.invoke({"input": "when is the initial release date of bitcoin", })

In [ ]:
response['output']

In [ ]:
response = agent_executor.invoke({"input": "what is the average prices for AAPL stock between 2023-01-01 and 2023-02-01? Use provided tools.", })

In [4]:
import os
#import openai
from langchain.tools import tool
import requests
from pydantic import BaseModel, Field
import datetime
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

In [ ]:
import panel as pn  # GUI
pn.extension()

import panel as pn
import param

In [ ]:
class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []

        #openAI
        
        # self.functions = [format_tool_to_openai_function(f) for f in tools]
        # self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)        
        # self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        # self.prompt = ChatPromptTemplate.from_messages([
        #     ("system", "You are helpful assistant"),
        #     MessagesPlaceholder(variable_name="chat_history"),
        #     ("user", "{input}"),
        #     MessagesPlaceholder(variable_name="agent_scratchpad")
        # ])
        # self.chain = RunnablePassthrough.assign(
        #     agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        # ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        # self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)

        #Ollama
        
        self.tools = tools
        self.model = ChatOllama(
            model="",
            temperature=0,
        ).bind_tools(self.tools)
          
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        
        self.prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are a helpful assistant.  \
                    Make sure to convert the user input date to YYYY-MM-DD format. \
                    Conver the use input token name to token ticker symbol, in lower case. \
                    For example, convert Bitcoin to btc and Ethereum to eth. ",
                ),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]
        )
        
        self.agent = create_tool_calling_agent(self.model, self.tools, self.prompt)   
        self.qa = AgentExecutor(agent=self.agent, tools=self.tools, verbose=False, memory=self.memory)

    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('Answer:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

tools = [wiki, get_stock_prices_and_volumes, calculate_var]

cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# Financial_QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard
inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# Financial_QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

